# Arrays and Summation

* $P_g$ Power Generation
* $C_g$ Cost of Power generation
* $L_d$ Load Points or Load demand

### Power Generation(Pg) Table 1 (gen tab in Spreadsheet)
|ID|Cost|Power Generation Limit|
|--|----|--------------|
|0|0.10|20kW|
|1|0.05|10kW|
|2|0.30|10kW|
|3|0.40|50 KW|
|4|0.01|5kW|

### Load Points (Pd) Table 2 (load tab in Spreadsheet)

|ID|Load Demand|
|--|----|
|0|50kW|
|1|20kW|
|2|30kW|

* Only generators 0 and 3 can provide power to load point 0

$$ \min  \sum_{i_g =0}^4 C_g(i_g)P_g(i_g) $$

S.t.

* Supply and demand must be equal
  
$$\sum_{i_g =0}^4 P_g(i_g) = \sum_{i_d =0}^2 P_d(i_d) $$ 

* Generators 0 and 3 supply power to load point 0

$$ P_d(0) \le P_g(0) + P_g(3) $$

* Each power generator should generate more than 0kW.
  $$ P_g(i_g) \ge 0 \qquad \forall i_g$$

* Each power generator have a power generation limit
  $$P_g(i_g) \le P_g(i_g)^{LIM} \qquad \forall i_g $$


Variable = $P_g(i_g)^{LIM}=\{20,10,40,50,50,5\}$
* As seen in Table 1

In [1]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd

In [3]:
#inputs
dataGen = pd.read_excel('1_inputs.xlsx', sheet_name='gen')
dataGen


,id,limit,cost
0,0,20,0.10
1,1,10,0.05
2,2,40,0.30
3,3,50,0.40
4,4,5,0.01


In [4]:
Ng = len(dataGen)
Ng # number of variables

5

In [7]:
dataGen.cost[1]

0.05

In [6]:
dataLoad = pd.read_excel('1_inputs.xlsx', sheet_name='load')
dataLoad

,id,value
0,0,50
1,1,20
2,2,30


In [14]:
#model
model = pyo.ConcreteModel()

### Learning

In [9]:
#model.Pg=pyo.Var()# lets just run this code
#model.Pg.pprint()

Pg : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :  None :  None :  None : False :  True :  Reals


In [10]:
#model.Pg=pyo.Var(bounds=(0,None))# lets just run this code
#model.Pg.pprint()

    'pyomo.core.base.var.SimpleVar'>) on block unknown with a new Component
    (type=<class 'pyomo.core.base.var.SimpleVar'>). This is usually indicative
    of a modelling error. To avoid this warning, use block.del_component() and
    block.add_component().
Pg : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :  None :  None : False :  True :  Reals


We have 5 generators, we could create, $P_{g1}, P_{g2}$ but instead, we can simply specify the dimensions.

In [17]:
Pg = model.Pg

### Dimensions 

* Sometimes we can have 2 dimensions Example: $5 \times 3=15 ~ variables$
$$ \sum_{i_g=0}^4 \sum_{t_g=0}^2 P_g(t_g)(i_g)$$

In [19]:
model = pyo.ConcreteModel()
model.Pg = pyo.Var(range(Ng), range(3), bounds=(0,None))
model.Pg.pprint()

Pg : Size=15, Index=Pg_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (0, 0) :     0 :  None :  None : False :  True :  Reals
    (0, 1) :     0 :  None :  None : False :  True :  Reals
    (0, 2) :     0 :  None :  None : False :  True :  Reals
    (1, 0) :     0 :  None :  None : False :  True :  Reals
    (1, 1) :     0 :  None :  None : False :  True :  Reals
    (1, 2) :     0 :  None :  None : False :  True :  Reals
    (2, 0) :     0 :  None :  None : False :  True :  Reals
    (2, 1) :     0 :  None :  None : False :  True :  Reals
    (2, 2) :     0 :  None :  None : False :  True :  Reals
    (3, 0) :     0 :  None :  None : False :  True :  Reals
    (3, 1) :     0 :  None :  None : False :  True :  Reals
    (3, 2) :     0 :  None :  None : False :  True :  Reals
    (4, 0) :     0 :  None :  None : False :  True :  Reals
    (4, 1) :     0 :  None :  None : False :  True :  Reals
    (4, 2) :     0 :  None :  None : False :  True :  Reals


### Define Variable

In [20]:
#model
model = pyo.ConcreteModel()
# variable
model.Pg = pyo.Var(range(Ng), bounds=(0,None))
Pg = model.Pg 
model.Pg.pprint()

Pg : Size=5, Index=Pg_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :  None :  None : False :  True :  Reals
      1 :     0 :  None :  None : False :  True :  Reals
      2 :     0 :  None :  None : False :  True :  Reals
      3 :     0 :  None :  None : False :  True :  Reals
      4 :     0 :  None :  None : False :  True :  Reals


### Contraints

* Power Balance thus total Generation == total Demand

$$\sum_{i_g =0}^4 P_g(i_g) = \sum_{i_d =0}^2 P_d(i_d) $$ 

In [23]:
dataLoad

,id,value
0,0,50
1,1,20
2,2,30


In [24]:
[dataLoad.value[d] for d in dataLoad.id]

[50, 20, 30]

In [26]:
[Pg[g] for g in dataGen.id] # same concept

In [28]:
dataGen.id

0    0
1    1
2    2
3    3
4    4
Name: id, dtype: int64

In [27]:
print(sum([Pg[g] for g in dataGen.id]))

Pg[0] + Pg[1] + Pg[2] + Pg[3] + Pg[4]


In [30]:
sum(Pg)# this just summed the id, but it will be replaced.

10

In [29]:
# The Load is already an array
sum(dataLoad.value)

100

In [31]:
pg_sum=sum([Pg[g] for g in dataGen.id])# summation of all Pg is position g
model.balance=pyo.Constraint(expr = pg_sum== sum(dataLoad.value) )

* Contraint 2.

$$ P_d(0) \le P_g(0) + P_g(3) $$

In [32]:
dataLoad.value[0]

50

In [33]:
## Always Put Optimization Variables before constaints
model.cond = pyo.Constraint(expr = Pg[0]+Pg[3] >= dataLoad.value[0])

In [35]:
model.pprint()

1 Set Declarations
    Pg_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    Pg : Size=5, Index=Pg_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals

2 Constraint Declarations
    balance : Size=1, Index=None, Active=True
        Key  : Lower : Body                                  : Upper : Active
        None : 100.0 : Pg[0] + Pg[1] + Pg[2] + Pg[3] + Pg[4] : 100.0 :   True
    cond : Size=1, Index=None, Active=True
        Key  : Lower : Body          : Upper : Active
        None :  50.0 : Pg[0] + Pg[3] :  +Inf :   True

4 Declarations: Pg_index Pg bal

* Upper Limit Contraints

$$P_g(i_g) \le P_g(i_g)^{LIM} \qquad \forall i_g $$

Which is list of constraints

In [36]:
model.limits =pyo.ConstraintList()
model.limits.pprint()

limits : Size=0, Index=limits_index, Active=True
    Key : Lower : Body : Upper : Active


In [37]:
for g in dataGen.id:
    model.limits.add(expr = Pg[g]<= dataGen.limit[g])

In [38]:
model.limits.pprint()

limits : Size=5, Index=limits_index, Active=True
    Key : Lower : Body  : Upper : Active
      1 :  -Inf : Pg[0] :  20.0 :   True
      2 :  -Inf : Pg[1] :  10.0 :   True
      3 :  -Inf : Pg[2] :  40.0 :   True
      4 :  -Inf : Pg[3] :  50.0 :   True
      5 :  -Inf : Pg[4] :   5.0 :   True


## Objective Function

$$ \min  \sum_{i_g =0}^4 C_g(i_g)P_g(i_g) $$


In [39]:
dataGen.cost

0    0.10
1    0.05
2    0.30
3    0.40
4    0.01
Name: cost, dtype: float64

In [40]:
cost_sum = sum([Pg[g]*dataGen.cost[g] for g in dataGen.id])
model.obj = pyo.Objective(expr=cost_sum)

## Define the solver

In [45]:
opt = SolverFactory('glpk')

In [46]:
results = opt.solve(model)

In [51]:
print(pyo.value(Pg[0])) # so thare are 5 values
print(pyo.value(Pg[1]))

20.0
10.0


In [52]:
# Create additional column
# create a list
dataGen['Pg'] = [pyo.value(Pg[g]) for g in dataGen.id]
dataGen

,id,limit,cost,Pg
0,0,20,0.10,20.0
1,1,10,0.05,10.0
2,2,40,0.30,35.0
3,3,50,0.40,30.0
4,4,5,0.01,5.0
